# JupyterLab Self-Hosted
An adventure of hosting ops.

## Where I started
Most of the tutorials for Jupyter involve bare-metal hosting using python virtual environments. A lot of people like using `conda` to manage the python environments and install packages. I found it to be somewhat limited compared to `pip`. I was able to isolate the `jupyter lab` server in a virtual environment, and I created a systemd service to start it automatically. But then I thought *why stop there?* I need it even more isolated. Why not use `docker`?

## Docker(ized) Deployment
The Jupyter organization has several off-the-shelf docker images available. They are steadily updated with the latest and greatest stuff. I also wanted to automate the deployment with my favorite docker tool, `docker compose`. This is hosted in the *ammocan* in a folder found at `home/samyules/docker-compose/JupyterLab`.

### Host Docker Compose File Tree

```text
samyules@ammocan:~/docker-compose/JupyterLab$ tree

│
├── data
│   ├── work   [ notebook storage ]
│   ├── ssh    [ Persistent ssh keys ]
│   └── config [ Persisitent git config --global ] 
├── compose.yaml
├── Dockerfile
└── requirements.txt
```
I updated the docker compose to reflect the final folder layout of the persistent volumes.

### Custom docker image with additional kernels

#### Dockerfile

```Dockerfile
FROM quay.io/jupyter/minimal-notebook:2023-12-25

ARG env_name=python310
ARG py_ver=3.10

# You can add additional libraries here
RUN mamba create --yes -p "${CONDA_DIR}/envs/${env_name}" \
    python=${py_ver} \
    'ipykernel' \
    'jupyterlab' && \
    mamba clean --all -f -y

# Create Python kernel and link it to jupyter
RUN "${CONDA_DIR}/envs/${env_name}/bin/python" -m ipykernel install --user --name="${env_name}" && \
    fix-permissions "${CONDA_DIR}" && \
    fix-permissions "/home/${NB_USER}"

# Any additional `pip` installs can be added by using the following line
# Using `mamba` is highly recommended though
COPY --chown=${NB_UID}:${NB_GID} requirements.txt /tmp/
RUN "${CONDA_DIR}/envs/${env_name}/bin/pip" install --no-cache-dir --requirement /tmp/requirements.txt && \
    fix-permissions "${CONDA_DIR}" && \
    fix-permissions "/home/${NB_USER}"

# This changes the custom Python kernel so that the custom environment will
# be activated for the respective Jupyter Notebook and Jupyter Console
# hadolint ignore=DL3059
RUN /opt/setup-scripts/activate_notebook_custom_env.py "${env_name}"

# Comment the line above and uncomment the section below instead to activate the custom environment by default
# Note: uncommenting this section makes "${env_name}" default both for Jupyter Notebook and Terminals
# More information here: https://github.com/jupyter/docker-stacks/pull/2047
USER root
RUN \
     # This changes a startup hook, which will activate the custom environment for the process
     echo conda activate "${env_name}" >> /usr/local/bin/before-notebook.d/10activate-conda-env.sh && \
     # This makes the custom environment default in Jupyter Terminals for all users which might be created later
     echo conda activate "${env_name}" >> /etc/skel/.bashrc && \
     # This makes the custom environment default in Jupyter Terminals for already existing NB_USER
     echo conda activate "${env_name}" >> "/home/${NB_USER}/.bashrc"

USER ${NB_UID}

# bash kernel install in ${env_name}
RUN "${CONDA_DIR}/envs/${env_name}/bin/python" -m bash_kernel.install
# finish sshkernel install
RUN "${CONDA_DIR}/envs/${env_name}/bin/python" -m sshkernel install --user
# RUN python -m bash_kernel.install

# Install ijavascript
# ijavascript build will fail without libzmq
# See GitHub issue https://github.com/n-riesco/ijavascript/issues/184
USER root
RUN apt-get update --yes && \
    apt-get upgrade --yes && \
    apt-get install -yq libzmq3-dev build-essential python2.7

RUN npm install -g ijavascript

USER ${NB_UID}
RUN ijsinstall

# rebuild jupyter lab to finish installing plugins
RUN jupyter lab build

EXPOSE 8888
CMD ["start-notebook.py", "--IdentityProvider.token=''", "--ServerApp.root_dir=/home/jovyan/work"]
```

#### requirements.txt

```text
pyautogen
jupyterlab-git
ansible-kernel
bash_kernel
sshkernel
nbclassic
jupyterlab-notifications
ntfy
```

#### compose.yaml

```yaml
services:
  web:
    build: .
    ports:
      - "10000:8888"
    volumes:
      - ./data/work:/home/jovyan/work
      - ./data/ssh:/home/jovyan/.ssh
      - ./data/config:/home/jovyan/.config
      - /etc/localtime:/etc/localtime:ro #sync clock with host
    environment:
      - TZ="America/Denver"
      - RESTARTABLE="yes"
    restart: always
```
The volumes section has been updated to place all of the persistent data inside of the data/ folder. Originally I only had pesistent storage of notebooks in the work/ folder, but in order to use the jupyterlab-git plugin I need persistent ssh keys and persistent git config files.

### Docker Compose commands

In [ ]:
# first time container build: run the  
# following command from the project folder
docker compose up

In [ ]:
# after making changes to the Dockerfile,
# compose.yaml, or requirements.txt run
docker compose build
docker compose up

## Additional Options

### Notifications
In the case of long-running processes, there is axtension available for JupyterLab that can send a notification when a process is complete. It is very creatively named: [juperterlab-notifications](https://github.com/mwakaba2/jupyterlab-notifications).

Browser Notification

<img src = "https://user-images.githubusercontent.com/3497137/118382531-3275eb80-b5bc-11eb-9810-5b92183609c3.png" align = "center" width = "300" >

Mobile notification with NTFY app

<img src = "https://user-images.githubusercontent.com/3497137/136384645-843b8496-ad40-4c89-998b-ff46ea9f73a7.png" align = "center" width = "300" >

#### Install
This extension can be installed wuth `pip`:

In [ ]:
pip install jupyterlab-notifications

### Settings
Use the following settings to update cell execution time for a notification and information to display in the notification. (in `Settings > Advanced Settings Editor`):
```yaml
{
  // Notifications
  // jupyterlab-notifications:plugin
  // Settings for the Notifications extension
  // ****************************************

  // Cell Number Type
  // Type of cell number to display when the report_cell_number is true. Select from 'cell_index' or ‘cell_execution_count'.
  cell_number_type: 'cell_index',

  // Enabled Status
  // Enable the extension or not.
  enabled: true,

  // Trigger only for the last selected notebook cell execution.
  // Trigger a notification only for the last selected executed notebook cell.
  // NOTE: Only Available in version >= v0.3.0
  last_cell_only: false,

  // Minimum Notebook Cell Execution Time
  // The minimum execution time to send out notification for a particular notebook cell (in seconds).
  minimum_cell_execution_time: 60,

  // Notification Methods
  // Option to send a notification with the specified method(s). The available options are 'browser' and 'ntfy'.
  notification_methods: ['browser'],

  // Report Notebook Cell Execution Time
  // Display notebook cell execution time in the notification.
  // If last_cell_only is set to true, the total duration of the selected cells will be displayed.
  report_cell_execution_time: true,

  // Report Notebook Cell Number
  // Display notebook cell number in the notification.
  report_cell_number: true
}
```

### Notifications using `ntfy`
You can recieve mobile notifications via `ntfy`

[ntfy 2.7.0 documentation](https://ntfy.readthedocs.io/en/latest/)
> ntfy brings notification to your shell. It can automatically provide desktop notifications when long running code executions finish or it can send push notifications to your phone when a specific execution finishes.

#### Enable notifications via `ntfy`
Install `ntfy`

In [ ]:
pip install ntfy

Create a configuration file for ntfy in `~/.config/ntfy/ntfy.yml`

Note: You will need to install the Pushover mobile app and create an account to generate your user key.
```yaml
backends:
  - pushover
pushover:
  user_key: YOUR_PUSHOVER_USER_KEY
  ```

Then change the `notification_methods` option to include `ntfy`.
```yaml
{
  // Notification Methods
  // Option to send a notification with the specified method(s). The available options are 'browser' and 'ntfy'.
  notification_methods: ['browser', 'ntfy'],
}
```
The `browser` method is for the standard browser notifications.